In [1]:
import numpy as np
import pandas as pd
import os
import math
os.getcwd()
os.chdir('C:\\Users\\EeLian\\Desktop\\OULAD_KAGGLE\\PythonCodeCleanUp\\Data')

### A01_Lean_Dashboard_for_EEE2014J: read data

In [2]:
# read data file.
# most of these can be filtered directly except assessments and studentVle

studentInfo = pd.read_csv('studentInfo.csv')
#EEE2014J
courses = pd.read_csv('courses.csv')
#EEE2014J
studentRegistration = pd.read_csv('studentRegistration.csv')
#EEE2014J
studentAssessment = pd.read_csv('studentAssessment.csv')
#EEE2014J
assessments = pd.read_csv('assessments.csv')
#HHH
studentVle = pd.read_csv('studentVle.csv')
#HHH
vle = pd.read_csv('vle.csv')
#EEE2014J
dates = pd.read_csv('dates.csv')
#two years date to date_groups

### A01_Lean_Dashboard_for_EEE2014J: date groups

In [3]:
dates['date_unregistration']=dates['date']
dates['date_registration']=dates['date']

dates['_date_group_unregistration'] = pd.Series([], dtype=float)
dates['_date_group_unregistration'] = dates['date_unregistration']
dates.loc[dates['date_unregistration'] <0, '_date_group_unregistration'] = dates['_date_group_unregistration'].apply(lambda x: math.ceil(float(x/7)))
dates.loc[dates['date_unregistration'] >=0, '_date_group_unregistration'] = dates['_date_group_unregistration'].apply(lambda x: math.floor(float((x/7)+1)))

dates['_date_group_registration'] = pd.Series([], dtype=float)
dates['_date_group_registration'] = dates['date_registration']
dates.loc[dates['date_registration'] <0, '_date_group_registration'] = dates['_date_group_registration'].apply(lambda x: math.ceil(float(x/7)))
dates.loc[dates['date_registration'] >=0, '_date_group_registration'] = dates['_date_group_registration'].apply(lambda x: math.floor(float((x/7)+1)))


### A01_Lean_Dashboard_for_EEE2014J: Table 1

In [4]:
# read datafile
studentInfo = pd.read_csv('studentInfo.csv')
courses = pd.read_csv('courses.csv')
studentRegistration = pd.read_csv('studentRegistration.csv')

# add _fl_latest_module_attended
studentInfo.sort_values(by= ['id_student','code_module','code_presentation'], inplace = True, ascending = False)
studentInfo['_fl_latest_module_attended'] = studentInfo.groupby(['id_student','code_module'])['id_student'].rank(method='first', ascending = False)

# merge/ left join to get studentInfo + courses + registration 
# name as studentInfo3 cos 3 files
studentInfo2 = studentInfo.merge(courses, how='left', left_on=['code_module', 'code_presentation'], right_on=['code_module', 'code_presentation'])
studentInfo3 = studentInfo2.merge(studentRegistration, how='left', left_on=['code_module', 'code_presentation', 'id_student'], right_on=['code_module', 'code_presentation', 'id_student'])

# grouping the dates as _date_groups of weeks based on registration and unregistration dates
# studentInfo3['_date_group_registration'] = pd.Series([], dtype=float)
# studentInfo3['_date_group_registration'] = studentInfo3['date_registration']
# studentInfo3.loc[studentInfo3['date_registration'].isnull(), '_date_group_registration'] = ''
# studentInfo3.loc[studentInfo3['date_registration'] <0, '_date_group_registration'] = studentInfo3['_date_group_registration'].apply(lambda x: math.ceil(float(x/7)))
# studentInfo3.loc[studentInfo3['date_registration'] >=0, '_date_group_registration'] = studentInfo3['_date_group_registration'].apply(lambda x: math.floor(float((x/7)+1)))

# studentInfo3['_date_group_unregistration'] = pd.Series([], dtype=float)
# studentInfo3['_date_group_unregistration'] = studentInfo3['date_unregistration']
# studentInfo3.loc[studentInfo3['date_unregistration'].isnull(), '_date_group_unregistration'] = ''
# studentInfo3.loc[studentInfo3['date_unregistration'] <0, '_date_group_unregistration'] = studentInfo3['_date_group_unregistration'].apply(lambda x: math.ceil(float(x/7)))
# studentInfo3.loc[studentInfo3['date_unregistration'] >=0, '_date_group_unregistration'] = studentInfo3['_date_group_unregistration'].apply(lambda x: math.floor(float((x/7)+1)))

# flag_actual_dropout
studentInfo3.loc[studentInfo3['date_unregistration'] >0, '_flag_actual_dropout'] = 1
studentInfo3.loc[studentInfo3['date_unregistration'] <=0, '_flag_actual_dropout'] = 0
studentInfo3.loc[studentInfo3['date_unregistration'].isnull(), '_flag_actual_dropout'] = 0

# _flag_prestart_withdrawal
studentInfo3.loc[((studentInfo3['_flag_actual_dropout']==0)&(studentInfo3['final_result'].isin(['Withdrawn']))), '_flag_prestart_withdrawal'] = 1
studentInfo3.loc[~((studentInfo3['_flag_actual_dropout']==0)&(studentInfo3['final_result'].isin(['Withdrawn']))), '_flag_prestart_withdrawal'] = 0

# _flag_final_result
studentInfo3['_flag_final_result'] = pd.Series([], dtype=int)
studentInfo3.loc[(studentInfo3['final_result'].isin(['Withdrawn']))|(studentInfo3['final_result'].isin(['Fail'])), '_flag_final_result'] = 1
studentInfo3.loc[(studentInfo3['final_result'].isin(['Distinction']))|(studentInfo3['final_result'].isin(['Pass'])), '_flag_final_result'] = 0

# update the imd_band without % sign
studentInfo3=studentInfo3.replace({
    'imd_band': {
        '10-20': '10-20%'
    }
})

# surrogate keys creation
studentInfo3['_surrogate_key_one']=\
    studentInfo3['id_student'].astype(str) + \
    studentInfo3['final_result']+ \
    studentInfo3['code_module']+ \
    studentInfo3['code_presentation']
studentInfo3['_surrogate_key_two']=\
    studentInfo3['final_result']+ \
    studentInfo3['code_module']+ \
    studentInfo3['code_presentation']
studentInfo3['_surrogate_key_three']=\
    studentInfo3['id_student'].astype(str) + \
    studentInfo3['code_module']+ \
    studentInfo3['code_presentation']
studentInfo3['_surrogate_key_four']=\
    studentInfo3['code_module']+ \
    studentInfo3['code_presentation']

# filter by EEE2014J only
LeanDashboard_Table1_EEE2014J_Profile= studentInfo3[(studentInfo3['_surrogate_key_four'].isin(['EEE2014J']))]

### A01_Lean_Dashboard_for_EEE2014J: Table 2

In [5]:
# merge studentAssessment with assessments table
studentAssessment2 = studentAssessment.merge(assessments, how='left', left_on=["id_assessment"], right_on=["id_assessment"])

# add _flag_null_score just to check if there is submission
studentAssessment2['_flag_null_score']=studentAssessment2['score'].isnull().astype(int)

# add weighted score
studentAssessment2['_weighted_score'] = studentAssessment2['score']*studentAssessment2['weight']

# add pass fail. the NaN means didnt submit
studentAssessment2.loc[studentAssessment2['score'] >=40, '_flag_assessment_passed'] = 1
studentAssessment2.loc[studentAssessment2['score'] <40, '_flag_assessment_passed'] = 0

# grouping the dates as _date_groups of weeks based on submission dates
studentAssessment2['_date_group'] = pd.Series([], dtype=float)
studentAssessment2['_date_group'] = studentAssessment2['date_submitted']
studentAssessment2.loc[studentAssessment2['date_submitted'] <0, '_date_group'] = studentAssessment2['_date_group'].apply(lambda x: math.ceil(float(x/7)))
studentAssessment2.loc[studentAssessment2['date_submitted'] >=0, '_date_group'] = studentAssessment2['_date_group'].apply(lambda x: math.floor(float((x/7)+1)))

# activity pre course 
studentAssessment2.loc[studentAssessment2['date_submitted'] <0, '_precourse'] = 1
studentAssessment2.loc[studentAssessment2['date_submitted'] >=0, '_precourse'] = 0

# distinct of student info
studentInfo3_results_distinct=studentInfo3[['id_student', 'code_module', 'code_presentation', 'final_result']]
studentInfo3_results_distinct=studentInfo3_results_distinct.drop_duplicates()

# merging assessment scores with student info
studentAssessment3 = studentAssessment2.merge(studentInfo3_results_distinct, how='left', \
                                                 left_on=['id_student', 'code_module', 'code_presentation'], \
                                                 right_on=['id_student', 'code_module', 'code_presentation'])

# surrogate keys creation
studentAssessment3['_surrogate_key_one']=\
    studentAssessment3['id_student'].astype(str) + \
    studentAssessment3['final_result']+ \
    studentAssessment3['code_module']+ \
    studentAssessment3['code_presentation']
studentAssessment3['_surrogate_key_two']=\
    studentAssessment3['final_result']+ \
    studentAssessment3['code_module']+ \
    studentAssessment3['code_presentation']
studentAssessment3['_surrogate_key_three']=\
    studentAssessment3['id_student'].astype(str) + \
    studentAssessment3['code_module']+ \
    studentAssessment3['code_presentation']
studentAssessment3['_surrogate_key_four']=\
    studentAssessment3['code_module']+ \
    studentAssessment3['code_presentation']

# division by the denominator for weighted score
studentAssessment3.loc[studentAssessment3['code_module'].isin(['EEE']), '_final_weighted_score'] = \
studentAssessment3['_weighted_score']/100

# filter by EEE2014J only
LeanDashboard_Table2_EEE2014J_Assessment= studentAssessment3[(studentAssessment3['_surrogate_key_four'].isin(['EEE2014J']))]

### A01_Lean_Dashboard_for_EEE2014J: Table 3

In [6]:
# merging of studentVle with vle table
studentVle3 = studentVle.merge(vle, how='left', \
                                                 left_on=["code_module", "code_presentation","id_site"], \
                                                 right_on=["code_module", "code_presentation","id_site"])

# grouping the dates as _date_groups of weeks based on activity dates
studentVle3['_date_group'] = pd.Series([], dtype=float)
studentVle3['_date_group'] = studentVle3['date']
studentVle3.loc[studentVle3['date'] <0, '_date_group'] = studentVle3['_date_group'].apply(lambda x: math.ceil(float(x/7)))
studentVle3.loc[studentVle3['date'] >=0, '_date_group'] = studentVle3['_date_group'].apply(lambda x: math.floor(float((x/7)+1)))

# identifying pre semester start or pre course start activities
studentVle3.loc[studentVle3['date'] <0, '_precourse'] = 1
studentVle3.loc[studentVle3['date'] >=0, '_precourse'] = 0

# distinct of student info
studentInfo3_results_distinct=studentInfo3[['id_student','code_module','code_presentation','final_result']]
studentInfo3_results_distinct=studentInfo3_results_distinct.drop_duplicates()

# merging vle with student info
studentVle4 = studentVle3.merge(studentInfo3_results_distinct, how='left', \
                                                 left_on=["id_student","code_module", "code_presentation"], \
                                                 right_on=["id_student","code_module", "code_presentation"])

# surrogate keys creation
studentVle4['_surrogate_key_one']=\
    studentVle4['id_student'].astype(str) + \
    studentVle4['final_result']+ \
    studentVle4['code_module']+ \
    studentVle4['code_presentation']
studentVle4['_surrogate_key_two']=\
    studentVle4['final_result']+ \
    studentVle4['code_module']+ \
    studentVle4['code_presentation']
studentVle4['_surrogate_key_three']=\
    studentVle4['id_student'].astype(str) + \
    studentVle4['code_module']+ \
    studentVle4['code_presentation']
studentVle4['_surrogate_key_four']=\
    studentVle4['code_module']+ \
    studentVle4['code_presentation']

# filter by EEE2014J only
LeanDashboard_Table3_EEE2014J_ClickStream= studentVle4[(studentVle4['_surrogate_key_four'].isin(['EEE2014J']))]

In [7]:
# find is banked tagging
Assessment_is_banked=LeanDashboard_Table2_EEE2014J_Assessment[LeanDashboard_Table2_EEE2014J_Assessment['is_banked'].isin([1])]
Assessment_is_banked2=Assessment_is_banked[['id_student','is_banked']]
Assessment_is_banked3=Assessment_is_banked2.drop_duplicates()
Assessment_is_banked3["_flag_isbanked"] = Assessment_is_banked3['is_banked']
Assessment_is_banked3=Assessment_is_banked3.drop(['is_banked'], axis = 1)

<ipython-input-7-458b5f7b0377>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Assessment_is_banked3["_flag_isbanked"] = Assessment_is_banked3['is_banked']


In [8]:
# send to csv
dates.to_csv('Dataset_Dashboard_Table1_Date_Group.csv',index=True,header=True)
LeanDashboard_Table1_EEE2014J_Profile.to_csv('Dataset_LeanDashboard_Table1_EEE2014J_Profile.csv',index=True,header=True)
LeanDashboard_Table2_EEE2014J_Assessment.to_csv('Dataset_LeanDashboard_Table2_EEE2014J_Assessment.csv',index=True,header=True)
LeanDashboard_Table3_EEE2014J_ClickStream.to_csv('Dataset_LeanDashboard_Table3_EEE2014J_ClickStream.csv',index=True,header=True)
Assessment_is_banked3.to_csv('Dataset_LeanDashboard_Table4_EEE2014J_isBanked.csv',index=True,header=True)

### Prepped Predictions

In [9]:
Dataset_Dashboard_Table1_Predictions = pd.read_csv('Dataset_Dashboard_Table1_Predictions.csv')
Dataset_Dashboard_Table1_Predictions.to_csv('Dataset_LeanDashboard_Table5_EEE2014J_Predictions.csv',index=True,header=True)

### Optimizing Baseline Models

* Refer to A007_EL_EEE_Modeling_Exclude_Churn for the optimized baseline models

### Other Dashboard Tables

* Dataset_Dashboard_Table1_GeoLatLong
* Dataset_Dashboard_Table1_HighestEd
* Dataset_Dashboard_Table1_ModuleDomain